In [ ]:
! pip install pytorch-lightning

In [ ]:
! pwd

In [ ]:
import pandas as pd
import os
import numpy as np
import matplotlib.pyplot as plt
from scipy.interpolate import interp1d
from sklearn.model_selection import GridSearchCV, train_test_split, KFold
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler
from datetime import datetime
from torch import nn
from torch import Tensor
from kmodule.keystroke_module import *
from kmodule.myMLP import *

In [ ]:
np.random.seed(42)

In [ ]:
X = np.load('/content/kmodule/data/nq_ex2_features_asym.npy')
Y = np.load('/content/kmodule/data/nq_ex2_gt_asym.npy')

In [ ]:
# normalizacja danych
scaler = MinMaxScaler()
X_norm = scaler.fit_transform(X)

In [ ]:
k_folds = KFold(n_splits=5, shuffle=True, random_state=42)
acc_scores = []

for k, (train, test) in enumerate(k_folds.split(X_norm)):
    print('Number of fold: ', k+1)

    X_train = X_norm[train, :]
    Y_train = Y[train]
    X_test = X_norm[test, :]
    Y_test = Y[test]

    # X_train, Y_train = sampling_imbalanced_data(X_norm[train, :], Y[train], opt='over')
    
    trainer,model = train_architecture(X_train, Y_train, X_norm.shape[1], max_epoch_train = 300)
    acc_val, df_rep = test_architecture(trainer, model, X_test, Y_test)
    acc_scores.append(acc_val)

    if k == 0:
      df_meas = df_rep
    else:
      df_meas = pd.concat([df_meas, df_rep],
                          axis=0, ignore_index=True)

print(df_meas.mean())
acc_scores = [round(elem, 2) for elem in acc_scores]
print("Cross Validation Accuracy Scores: ", acc_scores)
print("Average CV Score: ", np.mean(acc_scores))

In [ ]:
trainer, model = train_architecture(X, Y, X_norm.shape[1], max_epoch_train = 300)

In [ ]:
model = MLP.load_from_checkpoint('../models/model_mlp.ckpt',map_location=torch.device("cpu"),encoder_map_location=torch.device("cpu"))
model.eval()
Y = model(Tensor(va.reshape((1, -1))))
Y = np.round(np.squeeze(Y.detach().numpy()))